In [1]:
from QuantLib import *
from pandas import DataFrame

In [2]:
today = Date(27, January, 2011)
Settings.instance().evaluationDate = today

In [3]:
issueDate = Date(28, January, 2011)
maturity = Date(31, August, 2020)

schedule = Schedule(issueDate, maturity, Period(Semiannual),
                    UnitedStates(UnitedStates.GovernmentBond),
                    Unadjusted, Unadjusted,
                    DateGeneration.Backward, False)

bond = FixedRateBond(1, 100.0, schedule, [0.03625],
                     ActualActual(ActualActual.Bond),
                     Unadjusted, 100.0)

In [4]:
bond_yield = 0.034921

P1 = bond.dirtyPrice(bond_yield, bond.dayCounter(), Compounded, Semiannual)
print(P1)

101.07681646503603


In [5]:
flat_curve = FlatForward(bond.settlementDate(), bond_yield, ActualActual(ActualActual.Bond),
                         Compounded, Semiannual)

engine = DiscountingBondEngine(YieldTermStructureHandle(flat_curve))
bond.setPricingEngine(engine)
P2 = bond.dirtyPrice()

In [6]:
DataFrame([(P1, P2)], columns=['with yield', 'with curve'], index=[''])

,with yield,with curve
,101.076816,101.079986


In [7]:
DataFrame([(as_coupon(c).accrualStartDate(), as_coupon(c).accrualEndDate()) for c in bond.cashflows()[:-1]],
          columns=('start date', 'end date'),
          index= range(1, len(bond.cashflows())))

,start date,end date
1,"January 28th, 2011","February 28th, 2011"
2,"February 28th, 2011","August 31st, 2011"
3,"August 31st, 2011","February 29th, 2012"
4,"February 29th, 2012","August 31st, 2012"
5,"August 31st, 2012","February 28th, 2013"
6,"February 28th, 2013","August 31st, 2013"
7,"August 31st, 2013","February 28th, 2014"
8,"February 28th, 2014","August 31st, 2014"
9,"August 31st, 2014","February 28th, 2015"
10,"February 28th, 2015","August 31st, 2015"


In [8]:
dayCounter = ActualActual(ActualActual.Bond)

T = dayCounter.yearFraction(Date(28, 1, 2011), Date(28, 2, 2011),
                            Date(28, 8, 2010), Date(28, 2, 2011))

print(T)

0.08423913043478261


In [9]:
print(dayCounter.yearFraction(Date(28, 1, 2011), Date(28, 2, 2011),
                            Date(28, 2, 2010), Date(28, 2, 2011)))

0.08493150684931507


In [10]:
y = InterestRate(bond_yield, dayCounter, Compounded, Semiannual)
print(y.discountFactor(T))

0.997087920498809


### Yield-based calculation

In [11]:
data = []
for i, c in enumerate(bond.cashflows()[:-1]):
    c = as_coupon(c)
    A = c.amount()
    T = c.accrualPeriod()
    D = y.discountFactor(T)
    D_cumulative = D if i == 0 else D * data[-1][3]
    A_discounted = A * D_cumulative
    
    data.append((A, T, D, D_cumulative, A_discounted))
    
data.append((100, '', '', D_cumulative, 100*D_cumulative))
data = DataFrame(data, columns=('amount', 'T', 'discount', 'discount(cum.)', 'amount(disc.)'), index=['']*len(data))

print(data)

      amount          T  discount  discount(cum.)  amount(disc.)
    0.305367  0.0842391  0.997088        0.997088       0.304478
    1.812500        0.5  0.982839        0.979977       1.776208
    1.812500        0.5  0.982839        0.963160       1.745727
    1.812500        0.5  0.982839        0.946631       1.715769
    1.812500        0.5  0.982839        0.930386       1.686325
    1.812500        0.5  0.982839        0.914420       1.657386
    1.812500        0.5  0.982839        0.898728       1.628944
    1.812500        0.5  0.982839        0.883305       1.600990
    1.812500        0.5  0.982839        0.868146       1.573515
    1.812500        0.5  0.982839        0.853248       1.546513
    1.812500        0.5  0.982839        0.838606       1.519973
    1.812500        0.5  0.982839        0.824215       1.493889
    1.812500        0.5  0.982839        0.810070       1.468253
    1.812500        0.5  0.982839        0.796169       1.443056
    1.812500        0.5  

In [12]:
print(sum(data['amount(disc.)']))

101.07681646503603


In [13]:
print(bond.dirtyPrice(bond_yield, bond.dayCounter(), Compounded, Semiannual))

101.07681646503603


### Curve-based calculation

In [14]:
data = []
for c in bond.cashflows()[:-1]:
    A = c.amount()
    D = flat_curve.discount(c.date())
    A_discounted = A * D
    data.append((A, D, A_discounted))
data.append((100.0, D, 100.0*D))
data = DataFrame(data, columns=('amount', 'discount', 'amount(disc.)'), index=['']*len(data))

print(data)

      amount  discount  amount(disc.)
    0.305367  0.997119       0.304487
    1.812500  0.980008       1.776264
    1.812500  0.963190       1.745782
    1.812500  0.946661       1.715823
    1.812500  0.930415       1.686378
    1.812500  0.914449       1.657438
    1.812500  0.898756       1.628995
    1.812500  0.883332       1.601040
    1.812500  0.868174       1.573565
    1.812500  0.853275       1.546561
    1.812500  0.838632       1.520021
    1.812500  0.824240       1.493936
    1.812500  0.810096       1.468299
    1.812500  0.796194       1.443101
    1.812500  0.782530       1.418336
    1.812500  0.769102       1.393997
    1.812500  0.755903       1.370074
    1.812500  0.742931       1.346563
    1.812500  0.730182       1.323455
    1.812500  0.717651       1.300743
  100.000000  0.717651      71.765129


In [15]:
print(sum(data['amount(disc.)']))

101.0799861183387


In [17]:
T = dayCounter.yearFraction(Date(28, 1, 2011), Date(28, 2, 2011),
                            Date(28, 8, 2010), Date(28, 2, 2011))

P_y = bond.dirtyPrice(bond_yield, bond.dayCounter(), Compounded, Semiannual)
D_y = y.discountFactor(T)

P_c = bond.dirtyPrice()
D_c = flat_curve.discount(Date(28, 2, 2011))

print(P_y)
print(P_c * (D_y/D_c))

101.07681646503603
101.0768164650361
